In [7]:
!pip install opencv-python-headless

In [9]:
from IPython.display import display, Javascript
#from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from PIL import Image
import io
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'PIL'

In [ ]:
def capture_video_frame():
    # JS zum Aufnehmen eines Bildes mit der Webcam
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = '📸 Take a photo';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Create a Promise that resolves when capture is clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();

            return canvas.toDataURL('image/jpeg', 0.8);
        }
        takePhoto();
    ''')

    display(js)
    data = eval_js("takePhoto()")
    header, encoded = data.split(",", 1)
    img_bytes = b64decode(encoded)
    img = Image.open(io.BytesIO(img_bytes))
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

In [ ]:
from moravec import moravec_corner_detection

ModuleNotFoundError: No module named 'moravec'

In [ ]:
img = capture_video_frame()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

corners = moravec_corner_detection(gray)
corners_img = img.copy()
corners_img[corners > 0] = [0, 0, 255]  # Ecken rot markieren

# Anzeige
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1); plt.title("Original"); plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2); plt.title("Moravec Corners"); plt.imshow(cv2.cvtColor(corners_img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
edges = cv2.Canny(gray, 100, 200)

plt.imshow(edges, cmap='gray')
plt.title("Canny Edges")
plt.show()